In [ ]:
import os
import sys
from pathlib import Path

import numpy as np
import pandas as pd
import tensorflow as tf
import tensorflow_hub as hub
from huggingface_hub import from_pretrained_keras, push_to_hub_keras
from keras.models import load_model

from training_pipeline.data_generator import CropType, preprocess_image

sys.path.append("..")

MODEL_PATH = os.path.join(os.getcwd(), "../cache/keras_model/saved_model_rgb_classifier")
TEST_FILES_METADATA_CSV = "/Volumes/FD/ai_or_not/data/cifake/test/FAKE_metadata.csv"

REPO_ID = "ai-or-not/ai-or-not-model"

In [ ]:
model = tf.keras.models.load_model(MODEL_PATH)

In [ ]:
model.summary()

In [ ]:
metadata = pd.read_csv(TEST_FILES_METADATA_CSV, skiprows=1)
test_files = metadata.values
test_files = test_files[:10]

# extract prefix path from TEST_FILES_METADATA_CSV
prefx_path = Path(TEST_FILES_METADATA_CSV).parent
for file in test_files:
    file = file[0]
    image = preprocess_image(
        os.path.join(prefx_path, file),
        CropType.RESIZE,
    )

    print(f"File: {file} Image shape: {image.shape}")
    predictions = model.predict(np.array([image]))
    # take the argmax of the predictions
    max_index = np.argmax(predictions)

    print(f"Prediction: {predictions} Max: {max_index}")

In [ ]:
# Upload model to huggingface
push_to_hub_keras(
    model,
    repo_id=REPO_ID,
    tags=["0.1", "classification"],
)

In [ ]:
model = from_pretrained_keras(REPO_ID, token=True, force_download=True)

In [ ]:
prefx_path = Path(TEST_FILES_METADATA_CSV).parent
for file in test_files:
    file = file[0]
    image = preprocess_image(
        os.path.join(prefx_path, file),
        CropType.RESIZE,
    )

    print(f"File: {file} Image shape: {image.shape}")
    predictions = model(np.array([image]))
    # take the argmax of the predictions
    max_index = np.argmax(predictions)

    print(f"Prediction: {predictions} Max: {max_index}")